<a href="https://colab.research.google.com/github/sinchan-s/time-series-analysis/blob/main/tsa-greykite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install greykite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 22.5 MB 636 kB/s 
     |████████████████████████████████| 316 kB 70.6 MB/s 
     |████████████████████████████████| 164 kB 59.7 MB/s 
     |████████████████████████████████| 9.7 MB 54.0 MB/s 
     |████████████████████████████████| 9.4 MB 59.6 MB/s 
     |████████████████████████████████| 98 kB 8.1 MB/s 
     |████████████████████████████████| 1.9 MB 79.2 MB/s 
     |████████████████████████████████| 965 kB 67.9 MB/s 
     |████████████████████████████████| 295 kB 78.8 MB/s 
     |████████████████████████████████| 9.9 MB 43.7 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.1

In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly
%matplotlib inline

In [9]:
df = pd.read_csv("https://github.com/sinchan-s/time-series-analysis/raw/main/airline_passengers.csv")
df.dropna(axis=0, inplace=True)
df['Month'] = pd.to_datetime(df['Month'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Month                    144 non-null    datetime64[ns]
 1   Thousands of Passengers  144 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.4 KB


In [10]:
df.columns = ['month','#Passengers']
df['month'] = pd.to_datetime(df['month'],infer_datetime_format=True,format='%y%m')

In [11]:
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster 
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
# Specifies dataset information
metadata = MetadataParam(
     time_col="month",  # name of the time column
     value_col="#Passengers",  # name of the value column
     freq="MS"  #"MS" for Montly at start date
 )
forecaster = Forecaster()
result = forecaster.run_forecast_config(
     df=df,
     config=ForecastConfig(
         model_template=ModelTemplateEnum.SILVERKITE.name,
         forecast_horizon=100,  # forecasts 100 steps ahead
         coverage=0.95,  # 95% prediction intervals
         metadata_param=metadata
    )
)

/usr/local/lib/python3.8/dist-packages/greykite/framework/pipeline/pipeline.py:186: UserWarning:

Not enough training data to forecast the full forecast_horizon. Exercise extra caution with forecasted values after 72 periods.

/usr/local/lib/python3.8/dist-packages/greykite/framework/pipeline/pipeline.py:199: UserWarning:

test_horizon should be <= than 1/3 of the data set size to allow enough data to train a backtest model. Consider reducing to 48. If this is smaller than the forecast_horizon, you will need to make a trade-off between setting test_horizon=forecast_horizon and having enough data left over to properly train a realistic backtest model.

/usr/local/lib/python3.8/dist-packages/greykite/sklearn/cross_validation.py:191: UserWarning:

There are no CV splits under the requested settings. Decrease `forecast_horizon` and/or `min_train_periods`. Using default 90/10 CV split



Fitting 1 folds for each of 1 candidates, totalling 1 fits


/usr/local/lib/python3.8/dist-packages/greykite/algo/forecast/silverkite/forecast_silverkite.py:2803: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps

/usr/local/lib/python3.8/dist-packages/greykite/algo/forecast/silverkite/forecast_silverkite.py:2803: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps

/usr/local/lib/python3.8/dist-packages/greykite/algo/forecast/silverkite/forecast_silverkite.py:2803: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps

/usr/local/lib/python3.8/dist-packages/greykite/algo/forecast/silverkite/forecast_silverkite.py:2803: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps

/usr/local/lib/python3.8/dist-packages/greykite/algo/forecast/silverkite/forecast_silverkite.py:2803: UserWarning:

The granularity of data is large

In [13]:
ts = result.timeseries
fig = ts.plot()
plotly.io.show(fig)